In [36]:
import json

In [37]:
with open('test.json', 'r') as file:
    data = json.load(file)

In [38]:
for i, item in enumerate(data):
    item['id'] = i

In [39]:
unique_image_ids = set(item['image_id'] for item in data)
for unique_id in unique_image_ids:
    image_count = sum(1 for item in data if item['image_id'] == unique_id)
    for item in data:
        if item['image_id'] == unique_id:
            item['iscrowd'] = 1 if image_count > 1 else 0

In [40]:
for item in data:
    item['keypoints'][2::3] = [0 if test <= 0.17 else 2 for test in item['keypoints'][2::3]]

In [ ]:
def mirror_keypoints(keypoints, image_width):
    mirrored_keypoints = []
    for i in range(0, len(keypoints), 3):
        old_x = keypoints[i]
        new_x = image_width - old_x
        new_y = keypoints[i + 1]
        new_v = keypoints[i + 2]
        mirrored_keypoints.extend([new_x, new_y, new_v])
    return mirrored_keypoints

In [ ]:
def mirror_bbox(bbox, image_width):
    old_x, old_y, width, height = bbox
    new_x = image_width - (old_x + width)
    return [new_x, old_y, width, height]

In [ ]:
image_width = 463
mirrored_data = {}

In [41]:
for item in data:
    if '_m.png' in item['image_id']:
        original_image_id = item['image_id'].replace('_m.png', '.png')
        if original_image_id not in mirrored_data:
            mirrored_data[original_image_id] = []
        mirrored_keypoints = mirror_keypoints(item['keypoints'], image_width)
        mirrored_bbox = mirror_bbox(item['bbox'], image_width)
        mirrored_data[original_image_id].append({
            'keypoints': mirrored_keypoints,
            'bbox': mirrored_bbox
        })

In [ ]:
for item in data:
    if '_m.png' in item['image_id']:
        original_image_id = item['image_id'].replace('_m.png', '.png')
        if original_image_id in mirrored_data:
            mirrored_items = mirrored_data[original_image_id]
            item_index = data.index(item)
            if item_index < len(mirrored_items):
                item['keypoints'] = mirrored_items[item_index]['keypoints']
                item['bbox'] = mirrored_items[item_index]['bbox']

In [42]:
with open('test_corrected.json', 'w') as f:
    json.dump(data, f, indent=2)